## Setup

In [ ]:
import subprocess
import sys
from pathlib import Path

print(f"Python version: {sys.version}")
print(f"Working directory: {Path.cwd()}")

# Check if required directories exist
for dir_name in ['data', 'checkpoints', 'outputs', 'src']:
    dir_path = Path(dir_name)
    if dir_path.exists():
        print(f"✓ {dir_name}/ exists")
    else:
        print(f"✗ {dir_name}/ not found")
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"  Created {dir_name}/")

In [ ]:
# 의존성 설치 (필요시 실행)
INSTALL_DEPENDENCIES = False  # True로 변경하면 requirements.txt 설치

if INSTALL_DEPENDENCIES:
    print("Installing dependencies from requirements.txt...")
    result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✓ Dependencies installed successfully")
    else:
        print("✗ Installation failed")
        print(result.stderr)
else:
    print("⊘ Dependency installation skipped")
    print("  (Set INSTALL_DEPENDENCIES = True to install)")
    
# Check key packages
print("\nChecking key packages...")
required_packages = ['torch', 'transformers', 'sklearn', 'tqdm', 'numpy', 'matplotlib']
missing_packages = []

for package in required_packages:
    try:
        __import__(package)
        print(f"  ✓ {package}")
    except ImportError:
        print(f"  ✗ {package} (missing)")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠ Missing packages: {', '.join(missing_packages)}")
    print("  Set INSTALL_DEPENDENCIES = True and re-run this cell")
else:
    print("\n✓ All required packages are available")

## 의존성 설치 (선택사항)

첫 실행 시 필요한 패키지를 설치합니다.

## 전체 실행 설정

In [ ]:
# 실행할 단계 선택 (True/False)
RUN_CONFIG = {
    'preprocess': True,      # 1. 데이터 전처리
    'train_baseline': True,  # 2. Baseline 모델 학습
    'generate_text': True,   # 3. 대화 생성
    'train_multimodal': True,# 4. Multimodal 모델 학습
    'evaluate': True,        # 5. 평가
    
    # 평가 설정
    'n_eval_games': 1000,    # 평가할 게임 수
}

print("실행 설정:")
for key, value in RUN_CONFIG.items():
    print(f"  {key:20s}: {value}")

## Step 1: 데이터 전처리

Pluribus 데이터셋을 다운로드하고 377차원 feature로 변환합니다.

In [ ]:
if RUN_CONFIG['preprocess']:
    print("="*60)
    print("STEP 1: DATA PREPROCESSING")
    print("="*60)
    
    result = subprocess.run(['python', '1_preprocess_data.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 1 completed successfully")
    else:
        print("\n✗ Step 1 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Preprocessing failed")
else:
    print("⊘ Step 1 skipped")

## Step 2: Baseline 모델 학습

게임 상태만 사용하는 MLP 모델을 학습합니다.

In [ ]:
if RUN_CONFIG['train_baseline']:
    print("="*60)
    print("STEP 2: TRAIN BASELINE MODEL")
    print("="*60)
    
    result = subprocess.run(['python', '2_train_baseline.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 2 completed successfully")
    else:
        print("\n✗ Step 2 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Baseline training failed")
else:
    print("⊘ Step 2 skipped")

## Step 3: 대화 생성

LLM을 사용하여 각 게임 상태에 대한 대화를 생성합니다.

In [ ]:
if RUN_CONFIG['generate_text']:
    print("="*60)
    print("STEP 3: GENERATE DIALOGUES")
    print("="*60)
    
    result = subprocess.run(['python', '3_generate_dialogues.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 3 completed successfully")
    else:
        print("\n✗ Step 3 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Dialogue generation failed")
else:
    print("⊘ Step 3 skipped")

## Step 4: Multimodal 모델 학습

게임 상태와 대화를 결합한 multimodal 모델을 학습합니다.

In [ ]:
if RUN_CONFIG['train_multimodal']:
    print("="*60)
    print("STEP 4: TRAIN MULTIMODAL MODEL")
    print("="*60)
    
    result = subprocess.run(['python', '4_train_multimodal.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 4 completed successfully")
    else:
        print("\n✗ Step 4 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Multimodal training failed")
else:
    print("⊘ Step 4 skipped")

## Step 5: Rule-based Agent 평가

학습된 모델을 rule-based agent와 대결시켜 평가합니다.

In [ ]:
if RUN_CONFIG['evaluate']:
    print("="*60)
    print("STEP 5: EVALUATION")
    print("="*60)
    
    n_games = RUN_CONFIG['n_eval_games']
    
    # Evaluate baseline
    print("\n--- Evaluating Baseline Model ---")
    result_baseline = subprocess.run([
        'python', '5_evaluate_vs_rule_based.py',
        '--model_type', 'baseline',
        '--model_path', 'checkpoints/baseline_best.pt',
        '--n_games', str(n_games),
        '--output_dir', 'outputs'
    ], capture_output=False)
    
    if result_baseline.returncode != 0:
        print("\n✗ Baseline evaluation failed")
    
    # Evaluate multimodal
    print("\n--- Evaluating Multimodal Model ---")
    result_multimodal = subprocess.run([
        'python', '5_evaluate_vs_rule_based.py',
        '--model_type', 'multimodal',
        '--model_path', 'checkpoints/multimodal_best.pt',
        '--n_games', str(n_games),
        '--output_dir', 'outputs'
    ], capture_output=False)
    
    if result_multimodal.returncode != 0:
        print("\n✗ Multimodal evaluation failed")
    
    if result_baseline.returncode == 0 and result_multimodal.returncode == 0:
        print("\n✓ Step 5 completed successfully")
    else:
        print("\n⚠ Step 5 completed with errors")
else:
    print("⊘ Step 5 skipped")

## 결과 확인

In [ ]:
import json
import matplotlib.pyplot as plt

print("="*60)
print("EVALUATION RESULTS: Baseline vs Multimodal")
print("="*60)

# Load evaluation results
output_dir = Path('outputs')
baseline_file = output_dir / 'baseline_vs_rule_based.json'
multimodal_file = output_dir / 'multimodal_vs_rule_based.json'

if baseline_file.exists() and multimodal_file.exists():
    with open(baseline_file, 'r') as f:
        baseline = json.load(f)
    
    with open(multimodal_file, 'r') as f:
        multimodal = json.load(f)
    
    # Display comparison table
    print(f"\n{'Metric':<25s} {'Baseline':>15s} {'Multimodal':>15s} {'Diff':>12s}")
    print("=" * 70)
    
    # Total profit
    b_profit = baseline['total_profit']
    m_profit = multimodal['total_profit']
    diff = m_profit - b_profit
    diff_str = f"+{diff:.1f}" if diff > 0 else f"{diff:.1f}"
    print(f"{'Total Profit':<25s} {b_profit:>15.1f} {m_profit:>15.1f} {diff_str:>12s}")
    
    # Avg profit per hand
    b_avg = baseline['avg_profit_per_hand']
    m_avg = multimodal['avg_profit_per_hand']
    diff = m_avg - b_avg
    diff_str = f"+{diff:.2f}" if diff > 0 else f"{diff:.2f}"
    print(f"{'Avg Profit/Hand':<25s} {b_avg:>15.2f} {m_avg:>15.2f} {diff_str:>12s}")
    
    # Win rate
    b_wr = baseline['win_rate']
    m_wr = multimodal['win_rate']
    diff = m_wr - b_wr
    diff_str = f"+{diff:.1f}%" if diff > 0 else f"{diff:.1f}%"
    print(f"{'Win Rate':<25s} {b_wr:>14.1f}% {m_wr:>14.1f}% {diff_str:>12s}")
    
    print("-" * 70)
    
    # Win/Loss breakdown
    b_wins = baseline['wins']
    m_wins = multimodal['wins']
    b_losses = baseline['losses']
    m_losses = multimodal['losses']
    
    print(f"{'Wins':<25s} {b_wins:>15d} {m_wins:>15d} {m_wins - b_wins:>+12d}")
    print(f"{'Losses':<25s} {b_losses:>15d} {m_losses:>15d} {m_losses - b_losses:>+12d}")
    print(f"{'Games Played':<25s} {baseline['n_games']:>15d} {multimodal['n_games']:>15d}")
    
    print("\n" + "=" * 70)
    
    # Winner announcement
    if m_profit > b_profit:
        winner = "🏆 Multimodal"
        improvement = ((m_profit - b_profit) / abs(b_profit) * 100) if b_profit != 0 else 0
        print(f"{winner} wins with {improvement:+.1f}% profit improvement!")
    elif m_profit < b_profit:
        winner = "🏆 Baseline"
        improvement = ((b_profit - m_profit) / abs(m_profit) * 100) if m_profit != 0 else 0
        print(f"{winner} wins with {improvement:+.1f}% profit advantage!")
    else:
        print("🤝 It's a tie!")
    
    print("=" * 70)
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Plot 1: Total Profit
    ax = axes[0]
    models = ['Baseline', 'Multimodal']
    profits = [b_profit, m_profit]
    colors = ['#3498db', '#e74c3c']
    bars = ax.bar(models, profits, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax.set_ylabel('Total Profit', fontsize=12, fontweight='bold')
    ax.set_title('Total Profit Comparison', fontsize=13, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}',
                ha='center', va='bottom' if height > 0 else 'top',
                fontweight='bold', fontsize=11)
    
    # Plot 2: Win Rate
    ax = axes[1]
    win_rates = [b_wr, m_wr]
    bars = ax.bar(models, win_rates, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel('Win Rate (%)', fontsize=12, fontweight='bold')
    ax.set_title('Win Rate Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim([0, 100])
    ax.grid(axis='y', alpha=0.3)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom',
                fontweight='bold', fontsize=11)
    
    # Plot 3: Wins vs Losses
    ax = axes[2]
    x = np.arange(len(models))
    width = 0.35
    bars1 = ax.bar(x - width/2, [b_wins, m_wins], width, label='Wins', 
                   color='#2ecc71', alpha=0.7, edgecolor='black', linewidth=2)
    bars2 = ax.bar(x + width/2, [b_losses, m_losses], width, label='Losses', 
                   color='#e67e22', alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel('Count', fontsize=12, fontweight='bold')
    ax.set_title('Wins vs Losses', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(models)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('outputs/comparison_summary.png', dpi=150, bbox_inches='tight')
    print("\n📊 Visualization saved: outputs/comparison_summary.png")
    plt.show()
    
else:
    print("\n⚠ Evaluation results not found")
    print(f"  Expected files:")
    print(f"    - {baseline_file}")
    print(f"    - {multimodal_file}")
    print(f"\n  Make sure to run Step 5 (Evaluation) first.")

## 완료!

전체 파이프라인이 실행되었습니다.

### 생성된 파일들
- `data/processed/` - 전처리된 데이터
- `data/text/` - 생성된 대화
- `checkpoints/` - 학습된 모델
- `outputs/` - 평가 결과 및 시각화

### 다음 단계
- [evaluate_models.ipynb](evaluate_models.ipynb)에서 상세한 결과 분석 및 시각화
- 개별 스크립트를 직접 실행하여 파라미터 조정